In [35]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import pandas as pd
import numpy as np
import json
import io
import re
import string
from sklearn.metrics import f1_score, auc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [36]:
train_data = []
with io.open('train_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        train_data.append(d)

In [37]:
test_data = []
with io.open('test_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        test_data.append(d)

In [54]:
reds = ['т.е.', 'т.д.', 'т.к.', 'н.э.', 'т.п.', 'г.', 'гл.', 'пер.', 'гг.', 'ч.', 'вв.', 'ч.ч.', 'млн.', 'руб.', 'пп.',
        'др.', 'ст.', 'ул.', 'ч.', 'д.м.', 'в.', 'в. н. э.', 'вв', 'ред.', 'изм.', 'сокр.',
        'проф.', 'см.', ' п.', ' - ', 'ред.', 'тыс.', 'стр.', 'акад.', 'рис.']

def getFeatures(par, pos):
        cur_features = []
        par = par + '  '
        #mark
        cur_features.append(ord(par[pos]))
        for x in range(pos - 2, pos + 3):
            if x == pos:
                continue
            #space left, space right
            cur_features.append(par[x] == ' ')
            #punctuation left, punctuation right
            cur_features.append(par[x] in string.punctuation)
            #digit left, digit right
            cur_features.append(re.match('[0-9]', par[x]) is not None)
            #capital letter left, letter right
            cur_features.append(re.match('[A-ZА-Я]', par[x]) is not None)                        
            #lower letter left, letter right
            cur_features.append(re.match('[a-zа-я]', par[x]) is not None)            
            #open brackets left
            cur_features.append(par[x] == '(')            
            #open brackets right
            cur_features.append(par[x] == ')')    
        cur_features.append(int(par[pos + 1] == ' ') * 5)
        cur_features.append(int(re.match('[A-ZА-Я]', par[pos + 1]) is not None) * 7)
        #!!дата 17.05.2012
        cur_features.append(re.match('\w*[0-9]{2}.[0-9]{2}.[0-9]{4}\w*', par[(pos - 10):(pos + 10)]) is not None)
        cur_features.append(re.match('\w*[0-9]+-[0-9]+.\w*', par[(pos - 10):(pos + 10)]) is not None)
        subs = par[(pos - 5):(pos + 5)]
        for x in reds:
            cur_features.append(subs.lower().find(x) == -1)
        
        return cur_features

In [55]:
#mark, space left, space right, 
#punctuation left, punctuation right, 
#digit left, digit right, 
#letter left, letter right, 
#brackets left, brackets right, 
# стандартное сокращение?? неизвестное сокращение?
features = [] 
ans = []

for num, doc in enumerate(train_data):
    if (num % 1000 == 0):
        print(num, "/", len(train_data))
    par = doc['Paragraph'] + ' '
    end_pos = []
    for ss in doc['Sentences']:
        end_pos.append(par.find(ss) + len(ss) - 1)
    for pos in range(len(par) - 1):
        if par[pos] not in string.punctuation:
            continue        
        features.append(getFeatures(par, pos))
        ans.append(int(pos in end_pos))

0 / 22676
1000 / 22676
2000 / 22676
3000 / 22676
4000 / 22676
5000 / 22676
6000 / 22676
7000 / 22676
8000 / 22676
9000 / 22676
10000 / 22676
11000 / 22676
12000 / 22676
13000 / 22676
14000 / 22676
15000 / 22676
16000 / 22676
17000 / 22676
18000 / 22676
19000 / 22676
20000 / 22676
21000 / 22676
22000 / 22676


In [56]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features, ans)
print(clf)

pred = clf.predict(features).tolist()

print("f1 score = ", f1_score(ans, pred))

[46, False, False, False, False, True, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, 5, 0, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
f1 score =  0.990623979836


In [57]:
import codecs
out_data = []
fout = codecs.open("out", "w", "utf-8")
for num, doc in enumerate(test_data):
    par = doc['Paragraph'] + ' '
    for marks in doc['Marks']:        
        ind = marks['Index']
        pos = marks['Pos']
        mark = marks['Mark']        
        cur_features = getFeatures(par, pos)
        pred = clf.predict([cur_features])[0]
        tmp = par[(pos - 50):pos] + '!' + par[pos] + '!' + par[(pos + 1):(pos + 10)]        
        if (pred == 0):
            fout.write(tmp + "=" + str(pred) + "\n")
        out_data.append(pred)
fout.close()

In [58]:
print(len(out_data))
df = pd.DataFrame(out_data, columns=['Mark'], index=range(1,26477))
df.index.name = 'Id'
df.to_csv("sampleSubmission.csv")

26476


In [ ]:
акцию под лозунгом "С Дона выдачи нет, а с Днепра?!"! против в1
1 июня в 13–00, в Одинцовском суде, ул.Советская,7!.! Суд по м0
минск Московской области по адресу ул. Калинина, 4!.! 0
ороде Наро-Фоминск Московской области по адресу ул!.! Калинина1